### Vision2Clean AI

In [1]:
# 1️⃣ Install dependencies with CUDA-enabled PyTorch
# Run this once to set up your environment with CUDA support
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --quiet
%pip install ultralytics opencv-python matplotlib --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 2️⃣ Import libraries and check CUDA GPU
import torch
from ultralytics import YOLO
import os

print("=" * 60)
print("🚀 GPU & Environment Check")
print("=" * 60)

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ CUDA version: {torch.version.cuda}")
    print(f"✅ cuDNN version: {torch.backends.cudnn.version()}")
    print(f"✅ GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"   └─ GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"✅ Current device: cuda:{torch.cuda.current_device()}")
    device = "cuda"
else:
    print("⚠️  CUDA not available - will use CPU")
    print("   Run cell 1 to install CUDA-enabled PyTorch")
    device = "cpu"

print(f"\n🎯 Selected device: {device}")
print("=" * 60)

🚀 GPU & Environment Check
✅ PyTorch version: 2.5.1+cu121
✅ CUDA available: True
✅ CUDA version: 12.1
✅ cuDNN version: 90100
✅ GPU count: 1
   └─ GPU 0: NVIDIA GeForce RTX 4050 Laptop GPU
✅ Current device: cuda:0

🎯 Selected device: cuda


In [3]:
# 3️⃣ Define dataset YAML path
data_yaml = r"D:\Documents\Portfolio.github\Vision2Clean-AI\Dataset\data.yaml"

# Check YAML file
!type "{data_yaml}"

train: train/images
val: valid/images
test: test/images

nc: 5
names: ['Cardboard', 'Glass', 'Metal', 'Paper', 'Plastic']

roboflow:
  workspace: waste-segmentation-vlidl
  project: poster-dataset-ctglo
  version: 2
  license: CC BY 4.0
  url: https://universe.roboflow.com/waste-segmentation-vlidl/poster-dataset-ctglo/dataset/2


In [4]:
# 4️⃣ Load pretrained YOLOv11 segmentation model
model = YOLO("yolo11n-seg.pt")

# 5️⃣ Choose device
if torch.backends.mps.is_available():
    device = "mps"     # Apple Silicon GPU
elif torch.cuda.is_available():
    device = "cuda"    # NVIDIA GPU
else:
    device = "cpu"

print(f"🚀 Training on device: {device}")

🚀 Training on device: cuda


---
## 🎓 Training with Checkpoints

**Good news!** YOLO automatically saves your progress:

✅ **Automatic saves:**
- `last.pt` - Latest checkpoint (for resuming)
- `best.pt` - Best model based on validation accuracy
- Saved every 5 epochs + at the end

✅ **You can safely stop training:**
- Press **Interrupt** button or `Ctrl+C`
- All progress up to the last checkpoint is saved
- Resume anytime using Cell 6b

✅ **Checkpoint location:**
```
runs/segment/vision2clean_yolo11_seg/weights/
```

💡 **Pro tip:** If training is too slow, reduce `batch` size or `epochs` in Cell 6.

---

In [2]:
# 6️⃣ Start fine-tuning with automatic checkpoint saving
# YOLO automatically saves checkpoints:
#   - last.pt: Latest checkpoint (can resume from here)
#   - best.pt: Best model based on validation metrics
# You can stop training anytime (Ctrl+C) and resume later!

print("🎯 Starting training on device:", device)
print("📁 Checkpoints will be saved to: runs/segment/vision2clean_yolo11_seg/")
print("💾 Auto-saves: last.pt (resume) & best.pt (best model)")
print("⏸️  Stop anytime with Ctrl+C - progress is saved!\n")

model.train(
    data=data_yaml,          # Path to dataset YAML
    epochs=100,              # Total epochs
    imgsz=640,               # Input image size
    batch=8,                 # Reduce if RAM/GPU limited (try 16 if you have VRAM)
    device=device,           # CUDA / CPU
    project="runs/segment",
    name="vision2clean_yolo11_seg",
    
    # Checkpoint settings
    save=True,               # Save checkpoints
    save_period=5,           # Save checkpoint every 5 epochs (adjust as needed)
    
    # Performance optimizations for GPU
    cache=True,              # Cache images for faster training
    workers=4,               # Number of dataloader workers
    amp=True,                # Automatic Mixed Precision (faster on GPU)
    
    # Monitoring
    plots=True,              # Save training plots
    verbose=True             # Show detailed progress
)

NameError: name 'device' is not defined

In [ ]:
# 6️⃣b (OPTIONAL) Resume training from last checkpoint
# Run this cell ONLY if training was interrupted and you want to continue
# It will load the last checkpoint and continue from where it stopped

import os
import torch
from ultralytics import YOLO

# Re-detect device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"🎯 Device: {device}")

checkpoint_path = "runs/segment/vision2clean_yolo11_seg/weights/last.pt"

if os.path.exists(checkpoint_path):
    print(f"✅ Found checkpoint: {checkpoint_path}")
    print("🔄 Resuming training from last saved state...\n")
    
    # Load model from checkpoint
    model = YOLO(checkpoint_path)
    
    # Resume training (it will continue from the last epoch)
    model.train(
        resume=True,              # Resume from checkpoint
        device=device
    )
else:
    print(f"❌ No checkpoint found at: {checkpoint_path}")
    print("   Run cell 6 first to start training from scratch")

❌ No checkpoint found at: runs/segment/vision2clean_yolo11_seg/weights/last.pt
   Run cell 6 first to start training from scratch


In [ ]:
# 7️⃣ Evaluate model on validation set
# Run this after training completes to see final metrics

import torch
from ultralytics import YOLO

# Re-detect device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# Load the best trained model
model = YOLO("runs/segment/vision2clean_yolo11_seg/weights/best.pt")

print(f"🎯 Evaluating on device: {device}")
print("=" * 60)

# Run validation
metrics = model.val(device=device)

print("\n✅ Evaluation Results:")
print("=" * 60)
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print("=" * 60)

NameError: name 'model' is not defined

In [ ]:
# 8️⃣ Test on sample image


# 9️⃣ (Optional) Test live webcam
#results = model.predict(source=0, show=True)


In [ ]:
# =============================================================
# 🚀 Vision2Clean AI — Test Fine-Tuned YOLOv11-Seg with Webcam
# =============================================================

import cv2
import torch
from ultralytics import YOLO

# 1️⃣ Load your fine-tuned model
model = YOLO("runs/segment/vision2clean_yolo11_seg/weights/best.pt")

# 2️⃣ Choose the best device automatically
if torch.backends.mps.is_available():
    device = "mps"      # Apple Silicon GPU
elif torch.cuda.is_available():
    device = "cuda"     # NVIDIA GPU
else:
    device = "cpu"

print(f"✅ Using device: {device}")

# 3️⃣ Open laptop webcam (0 = default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Could not access webcam.")
    raise SystemExit()

# 4️⃣ Live detection loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv11-Seg inference
    results = model(frame, device=device, verbose=False)[0]

    # Draw masks + labels on the frame
    annotated = results.plot()

    # Display in window
    cv2.imshow("Vision2Clean AI — Live Waste Detection", annotated)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 5️⃣ Cleanup
cap.release()
cv2.destroyAllWindows()
print("👋 Stream closed.")
